In [15]:
pip install flask tensorflow pillow numpy

In [16]:
!pip install flask-ngrok pyngrok

In [14]:
from pyngrok import ngrok
from flask import Flask, request, jsonify
import tensorflow as tf, numpy as np, io, base64
from PIL import Image
import sqlite3
import os
import io
import base64
import datetime
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)


Mounted at /content/drive/


In [17]:
# Cell 2: Khởi tạo CSDL (Chạy cell này 1 lần thủ công)

# Đảm bảo bạn đã tạo thư mục "SmartTrash" trong Google Drive của mình
DRIVE_PATH = '/content/drive/MyDrive/SmartTrash'
DB_PATH = os.path.join(DRIVE_PATH, 'waste_log.db')
IMAGE_DIR = os.path.join(DRIVE_PATH, 'images')

# Tạo các thư mục nếu chưa có
os.makedirs(IMAGE_DIR, exist_ok=True)

# Kết nối và tạo bảng
try:
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Danh sách các loại rác (giống hệt trong Node-RED)
    tables = ["paper", "cardboard", "metal", "plastic", "glass", "trash"]

    for table in tables:
        cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {table} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            filepath TEXT,
            label TEXT,
            confidence REAL
        )
        ''')
        print(f"Table '{table}' checked/created.")

    conn.commit()
    conn.close()
    print("Database setup complete.")

except Exception as e:
    print(f"Lỗi khi setup CSDL: {e}")

Table 'paper' checked/created.
Table 'cardboard' checked/created.
Table 'metal' checked/created.
Table 'plastic' checked/created.
Table 'glass' checked/created.
Table 'trash' checked/created.
Database setup complete.


In [18]:
model_path = '/content/drive/MyDrive/model ai/garbage_classifier.h5'

In [19]:
from tensorflow.keras.layers import DepthwiseConv2D

class PatchedDepthwiseConv2D(DepthwiseConv2D):
    def __init__(self, *args, groups=1, **kwargs):
        super().__init__(*args, **kwargs)

custom_objects = {'DepthwiseConv2D': PatchedDepthwiseConv2D}

import tensorflow as tf
model = tf.keras.models.load_model(model_path, custom_objects=custom_objects, compile=False)


In [ ]:
# !pip install pyngrok
ngrok.set_auth_token("35N5tLOo2zRUAbwP1Bs57ShxFPw_5aSUaYstxiAJAUnj9brAK")
app = Flask(__name__)

# load model
# model = tf.keras.models.load_model(model_path, compile=False)
classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

def preprocess(img):
    img = img.resize((224, 224))
    img = np.expand_dims(np.array(img) / 255.0, axis=0)
    return img

app = Flask(__name__)

# Lấy đường dẫn từ cell trước
DRIVE_PATH = '/content/drive/MyDrive/SmartTrash'
DB_PATH = os.path.join(DRIVE_PATH, 'waste_log.db')
IMAGE_DIR = os.path.join(DRIVE_PATH, 'images')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        if 'image' not in data:
            return jsonify({'error': 'Missing image data'}), 400

        # 1. Giải mã ảnh (Dùng cho AI và Lưu trữ)
        img_bytes = base64.b64decode(data['image'])

        # 2. Chạy AI (Như cũ)
        img = Image.open(io.BytesIO(img_bytes)).convert('RGB')
        # (Thay thế bằng hàm preprocess của bạn)
        preprocessed_img = preprocess(img)
        preds = model.predict(preprocessed_img)[0]

        idx = np.argmax(preds)
        label = classes[idx]
        confidence = float(preds[idx])

        # 3. (MỚI) Lưu ảnh vào Google Drive
        ts = datetime.datetime.now()
        file_timestamp = ts.strftime('%Y-%m-%dT%H-%M-%S')

        # Tạo thư mục con (ví dụ: .../images/paper)
        label_image_dir = os.path.join(IMAGE_DIR, label)
        os.makedirs(label_image_dir, exist_ok=True)

        # Đường dẫn file trên Drive
        file_path = os.path.join(label_image_dir, f"{file_timestamp}_{label}.jpg")

        # Ghi file ảnh
        with open(file_path, 'wb') as f:
            f.write(img_bytes)

        # 4. (MỚI) Lưu CSDL vào Google Drive
        db_timestamp = ts.isoformat()
        try:
            conn = sqlite3.connect(DB_PATH)
            cursor = conn.cursor()
            # Dùng tên bảng động (label)
            cursor.execute(f"INSERT INTO {label} (timestamp, filepath, label, confidence) VALUES (?, ?, ?, ?)",
                           (db_timestamp, file_path, label, confidence))
            conn.commit()
            conn.close()
        except Exception as e:
            print(f"Lỗi ghi CSDL: {e}") # Colab sẽ in lỗi này ra console

        # 5. Trả kết quả về Node-RED (Như cũ)
        return jsonify({
            "label": label,
            "confidence": confidence
        })

    except Exception as e:
        print(f"Lỗi trong /predict: {e}")
        return jsonify({'error': str(e)}), 500

# Mở tunnel ngrok và in URL public
public_url = ngrok.connect(5000)
print("🌍 Public URL:", public_url)
app.run(port=5000)

🌍 Public URL: NgrokTunnel: "https://unomnipotent-hisako-granularly.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


INFO:werkzeug:127.0.0.1 - - [18/Nov/2025 05:53:21] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step


...

INFO:werkzeug:127.0.0.1 - - [18/Nov/2025 06:15:43] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


INFO:werkzeug:127.0.0.1 - - [18/Nov/2025 06:16:20] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


INFO:werkzeug:127.0.0.1 - - [18/Nov/2025 06:16:34] "POST /predict HTTP/1.1" 200 -


In [ ]:
print(tf.__version__)

In [ ]:
import h5py
with h5py.File(model_path, 'r') as f:
    print(f.attrs.get('keras_version'))
